In [5]:
!pip install --upgrade pip --quiet

In [6]:
!pip install -U transformers accelerate fpdf2 --quiet

In [7]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
import difflib
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
from PIL import Image
import requests
import torch

model_id = "google/gemma-3-12b-it"

model = Gemma3ForConditionalGeneration.from_pretrained(
    model_id, device_map="auto"
).eval()

processor = AutoProcessor.from_pretrained(model_id)

messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant that converts SQL DDL into DBML format."}]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """Convert the following SQL DDL into DBML format:

```sql
CREATE TABLE public.customer_address (
    address_id text NOT NULL,
    customer_id text NOT NULL,
    address_type text NOT NULL,
    street varchar(255) NOT NULL,
    city varchar(100) NOT NULL,
    state varchar(100) NULL,
    postal_code varchar(20) NULL,
    country varchar(100) NOT NULL,
    is_primary bool DEFAULT false NOT NULL,
    CONSTRAINT customer_address_pkey PRIMARY KEY (address_id)
);

CREATE INDEX idx_customer_address_customer_id ON public.customer_address USING btree (customer_id);
CREATE INDEX idx_customer_address_type ON public.customer_address USING btree (address_type);

ALTER TABLE public.customer_address ADD CONSTRAINT fk_customer
FOREIGN KEY (customer_id) REFERENCES public.customers(customer_id);
""" }
        ]
    }
]


inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=512, do_sample=False)
    generation = generation[0][input_len:]

decoded = processor.decode(generation, skip_special_tokens=True)
print(decoded)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.95` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:653: UserWarning: `do_sample` is set to `False`. However, `top_k` is set to `64` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_k`.
  warnings.warn(


```dbml
Table customer_address {
  address_id text [PK, not null]
  customer_id text [not null]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Index idx_customer_address_customer_id {
    customer_id
  }

  Index idx_customer_address_type {
    address_type
  }
}

Ref: customer_address.customer_id > customers.customer_id
```


In [ ]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

expected_output = """
Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [not null, default: false]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }
}

Ref: customer_address.customer_id > customers.customer_id
"""

# Format for chat model
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant that converts SQL DDL into DBML format."}]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """Convert the following SQL DDL into DBML format this is the expected format {}:

```sql
CREATE TABLE public.customer_address (
    address_id text NOT NULL,
    customer_id text NOT NULL,
    address_type text NOT NULL,
    street varchar(255) NOT NULL,
    city varchar(100) NOT NULL,
    state varchar(100) NULL,
    postal_code varchar(20) NULL,
    country varchar(100) NOT NULL,
    is_primary bool DEFAULT false NOT NULL,
    CONSTRAINT customer_address_pkey PRIMARY KEY (address_id)
);

CREATE INDEX idx_customer_address_customer_id ON public.customer_address USING btree (customer_id);
CREATE INDEX idx_customer_address_type ON public.customer_address USING btree (address_type);

ALTER TABLE public.customer_address ADD CONSTRAINT fk_customer
FOREIGN KEY (customer_id) REFERENCES public.customers(customer_id);
""" }
        ]
    }
]

# Tokenize input
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

# Generate + time it
start_time = time.time()
with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=512, do_sample=False)
end_time = time.time()

# Process output
generation = generation[0][input_len:]
decoded_output = processor.decode(generation, skip_special_tokens=True)

# Evaluation
generation_time = end_time - start_time
tokens_generated = generation.shape[-1]
tokens_per_second = tokens_generated / generation_time
cost = (generation_time / SECONDS_PER_HOUR) * GPU_COST_PER_HOUR

# Accuracy via fuzzy matching
similarity = difflib.SequenceMatcher(None, expected_output.strip(), decoded_output.strip()).ratio()

# Efficiency metrics
accuracy_per_dollar = similarity / cost
speed_per_dollar = tokens_per_second / cost

# Results
print("\n=== Generated Output ===\n")
print(decoded_output)

print("\n=== Evaluation Metrics ===")
print(f"Tokens Generated      : {tokens_generated}")
print(f"Generation Time       : {generation_time:.2f} sec")
print(f"Speed                 : {tokens_per_second:.2f} tokens/sec")
print(f"Cost (A100 @ $1.89/hr): ${cost:.6f}")
print(f"Accuracy (fuzzy)      : {similarity:.4f}")
print(f"Efficiency - Accuracy per $1 : {accuracy_per_dollar:.2f}")
print(f"Efficiency - Speed per $1    : {speed_per_dollar:.2f} tokens/sec per $1")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



=== Generated Output ===

```dbml
Table customer_address {
  address_id text [PK, not null]
  customer_id text [not null]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Index idx_customer_address_customer_id {
    customer_id
  }

  Index idx_customer_address_type {
    address_type
  }
}

Ref: customer_address.customer_id > customers.customer_id
```

=== Evaluation Metrics ===
Tokens Generated      : 174
Generation Time       : 8.73 sec
Speed                 : 19.94 tokens/sec
Cost (A100 @ $1.89/hr): $0.004581
Accuracy (fuzzy)      : 0.7741
Efficiency - Accuracy per $1 : 168.99
Efficiency - Speed per $1    : 4353.24 tokens/sec per $1


In [ ]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

expected_output = """
Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary boolean [not null, default: false]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }
}
"""
expected_format = """
Project BankingSystem {
  database_type: "PostgreSQL"
  note: "Comprehensive sample DBML schema"
}

Enum account_status {
  Active
  Inactive
  Closed
  Suspended
}

Enum transaction_type {
  Deposit
  Withdrawal
  Transfer
  Fee
}

Table customers {
  customer_id uuid [pk, not null]
  first_name varchar(50) [not null]
  last_name varchar(50) [not null]
  email varchar(100) [unique, not null]
  phone_number varchar(20)
  date_of_birth date
  created_at timestamp
  updated_at timestamp

  Indexes {
    (email) [unique, name: "idx_customers_email"]
  }

  Note: "Stores customer details"
}

Table branches {
  branch_id uuid [pk, not null]
  branch_name varchar(100) [not null]
  location varchar(100)
  is_active boolean [default: true]
  created_at timestamp [default: `CURRENT_TIMESTAMP`]
  updated_at timestamp

  Note: "Bank branch locations"
}

Table accounts {
  account_id uuid [pk, not null]
  customer_id uuid [not null, ref: > customers.customer_id]
  branch_id uuid [ref: > branches.branch_id]
  account_type varchar(50) [not null]
  balance numeric(18, 2)
  interest_rate numeric(5, 2)
  open_date date
  status account_status [not null]
  created_at timestamp
  updated_at timestamp

  Indexes {
    (customer_id) [name: "idx_accounts_customer_id"]
    (status) [name: "idx_accounts_status"]
    (account_type, status) [name: "idx_accounts_type_status"]
  }

  Note: "Accounts held by customers"
}

Table transactions {
  transaction_id uuid [pk, not null]
  account_id uuid [not null, ref: > accounts.account_id]
  txn_type transaction_type [not null]
  amount numeric(18, 2) [not null]
  currency_code char(3)
  transaction_date timestamp
  description text

  Indexes {
    (account_id) [name: "idx_transactions_account_id"]
    (transaction_date) [name: "idx_transactions_date"]
  }

  Note: "Transaction records for accounts"
}

"""
# Format for chat model
messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant that converts SQL DDL into DBML format."}]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": f"""Convert the following SQL DDL into DBML format.
Use the structure shown below as the expected format:

{expected_format}
"""
            },
            {
                "type": "text",
                "text": """```sql
CREATE TABLE public.customer_address (
    address_id text NOT NULL,
    customer_id text NOT NULL,
    address_type text NOT NULL,
    street varchar(255) NOT NULL,
    city varchar(100) NOT NULL,
    state varchar(100) NULL,
    postal_code varchar(20) NULL,
    country varchar(100) NOT NULL,
    is_primary bool DEFAULT false NOT NULL,
    CONSTRAINT customer_address_pkey PRIMARY KEY (address_id)
);

CREATE INDEX idx_customer_address_customer_id ON public.customer_address USING btree (customer_id);
CREATE INDEX idx_customer_address_type ON public.customer_address USING btree (address_type);

ALTER TABLE public.customer_address ADD CONSTRAINT fk_customer
FOREIGN KEY (customer_id) REFERENCES public.customers(customer_id);
"""
            }
        ]
    }
]

# Tokenize input
inputs = processor.apply_chat_template(
    messages, add_generation_prompt=True, tokenize=True,
    return_dict=True, return_tensors="pt"
).to(model.device, dtype=torch.bfloat16)

input_len = inputs["input_ids"].shape[-1]

# Generate + time it
start_time = time.time()
with torch.inference_mode():
    generation = model.generate(**inputs, max_new_tokens=512, do_sample=False)
end_time = time.time()

# Process output
generation = generation[0][input_len:]
decoded_output = processor.decode(generation, skip_special_tokens=True)

# Evaluation
generation_time = end_time - start_time
tokens_generated = generation.shape[-1]
tokens_per_second = tokens_generated / generation_time
cost = (generation_time / SECONDS_PER_HOUR) * GPU_COST_PER_HOUR

# Accuracy via fuzzy matching
similarity = difflib.SequenceMatcher(None, expected_output.strip(), decoded_output.strip()).ratio()

# Efficiency metrics
accuracy_per_dollar = similarity / cost
speed_per_dollar = tokens_per_second / cost

# Results
print("\n=== Generated Output ===\n")
print(decoded_output)

print("\n=== Evaluation Metrics ===")
print(f"Tokens Generated      : {tokens_generated}")
print(f"Generation Time       : {generation_time:.2f} sec")
print(f"Speed                 : {tokens_per_second:.2f} tokens/sec")
print(f"Cost (A100 @ $1.89/hr): ${cost:.6f}")
print(f"Accuracy (fuzzy)      : {similarity:.4f}")
print(f"Efficiency - Accuracy per $1 : {accuracy_per_dollar:.2f}")
print(f"Efficiency - Speed per $1    : {speed_per_dollar:.2f} tokens/sec per $1")


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.



=== Generated Output ===

```dbml
Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}
```

=== Evaluation Metrics ===
Tokens Generated      : 182
Generation Time       : 12.01 sec
Speed                 : 15.15 tokens/sec
Cost (A100 @ $1.89/hr): $0.006306
Accuracy (fuzzy)      : 0.9062
Efficiency - Accuracy per $1 : 143.69
Efficiency - Speed per $1    : 2402.73 tokens/sec per $1


In [ ]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

ddl_inputs = [
"""
CREATE TABLE public.customer_address (
    address_id text NOT NULL,
    customer_id text NOT NULL,
    address_type text NOT NULL,
    street varchar(255) NOT NULL,
    city varchar(100) NOT NULL,
    state varchar(100) NULL,
    postal_code varchar(20) NULL,
    country varchar(100) NOT NULL,
    is_primary bool DEFAULT false NOT NULL,
    CONSTRAINT customer_address_pkey PRIMARY KEY (address_id)
);
CREATE INDEX idx_customer_address_customer_id ON public.customer_address USING btree (customer_id);
CREATE INDEX idx_customer_address_type ON public.customer_address USING btree (address_type);
ALTER TABLE public.customer_address ADD CONSTRAINT fk_customer FOREIGN KEY (customer_id) REFERENCES public.customers(customer_id);
"""
    ,
"""
CREATE TABLE public.customers (
	customer_id text NOT NULL,
	entity_individual varchar(20) NULL,
	legal_company_name varchar(100) NULL,
	company_trade_name varchar(100) NULL,
	full_legal_name varchar(100) NULL,
	name_prefix varchar(10) NULL,
	first_name varchar(100) NULL,
	middle_name varchar(100) NULL,
	last_name varchar(100) NULL,
	alias varchar(100) NULL,
	date_of_birth text NULL,
	date_of_incorporation text NULL,
	national_id text NULL,
	country_of_legal_formation varchar(100) NULL,
	country_of_birth varchar(100) NULL,
	country_of_principle_business varchar(100) NULL,
	country_of_primary_residence varchar(100) NULL,
	tax_id text NULL,
	age int4 NULL,
	gender text NULL,
	ethnicity varchar(100) NULL,
	nationality varchar(100) NULL,
	citizenship1 varchar(100) NULL,
	citizenship2 varchar(100) NULL,
	native_language varchar(50) NULL,
	religion varchar(100) NULL,
	marital_status text NULL,
	dependents_count int4 NULL,
	household_income numeric(15, 2) NULL,
	education_level text NULL,
	employment_status text NULL,
	occupation varchar(100) NULL,
	industry varchar(100) NULL,
	company_name varchar(255) NULL,
	years_with_employer int4 NULL,
	job_role varchar(100) NULL,
	annual_income numeric(15, 2) NULL,
	primary_bank_relation text NULL,
	credit_score numeric(5, 2) NULL,
	home_ownership text NULL,
	residence_duration int4 NULL,
	vehicle_ownership bool NULL,
	financial_dependents int4 NULL,
	preferred_currency varchar(10) NULL,
	preferred_branch varchar(100) NULL,
	preferred_contact_method text NULL,
	social_media_presence jsonb NULL,
	risk_score numeric(5, 2) NULL,
	created_at timestamp NULL,
	updated_at timestamp NULL,
	CONSTRAINT customers_pkey PRIMARY KEY (customer_id)
);
CREATE INDEX idx_customer_company ON public.customers USING btree (company_name);
CREATE INDEX idx_customer_credit ON public.customers USING btree (credit_score);
CREATE INDEX idx_customer_name ON public.customers USING btree (last_name, first_name);
CREATE INDEX idx_customer_national_id ON public.customers USING btree (national_id);
CREATE INDEX idx_customer_risk ON public.customers USING btree (risk_score);
CREATE INDEX idx_customer_tax_id ON public.customers USING btree (tax_id);
"""
]

for i in ddl_inputs:
    messages = [
    {
        "role": "system",
        "content": [{"type": "text", "text": "You are a helpful assistant that converts SQL DDL into DBML format."}]
    },
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": """Convert the following SQL DDL into DBML format.
                        Use the structure shown below as the expected format
                        Project BankingSystem {
                          database_type: PostgreSQL
                          note: "Comprehensive sample DBML schema"
                        }

                        Enum account_status {
                          Active
                          Inactive
                          Closed
                          Suspended
                        }

                        Enum transaction_type {
                          Deposit
                          Withdrawal
                          Transfer
                          Fee
                        }

                        Table customers {
                          customer_id uuid [pk, not null]
                          first_name varchar(50) [not null]
                          last_name varchar(50) [not null]
                          email varchar(100) [unique, not null]
                          phone_number varchar(20)
                          date_of_birth date
                          created_at timestamp
                          updated_at timestamp

                          Indexes {
                            (email) [unique, name: "idx_customers_email"]
                          }

                          Note: "Stores customer details"
                        }

                        Table branches {
                          branch_id uuid [pk, not null]
                          branch_name varchar(100) [not null]
                          location varchar(100)
                          is_active boolean [default: true]
                          created_at timestamp [default: `CURRENT_TIMESTAMP`]
                          updated_at timestamp

                          Note: "Bank branch locations"
                        }

                        Table accounts {
                          account_id uuid [pk, not null]
                          customer_id uuid [not null, ref: > customers.customer_id]
                          branch_id uuid [ref: > branches.branch_id]
                          account_type varchar(50) [not null]
                          balance numeric(18, 2)
                          interest_rate numeric(5, 2)
                          open_date date
                          status account_status [not null]
                          created_at timestamp
                          updated_at timestamp

                          Indexes {
                            (customer_id) [name: "idx_accounts_customer_id"]
                            (status) [name: "idx_accounts_status"]
                            (account_type, status) [name: "idx_accounts_type_status"]
                          }

                          Note: "Accounts held by customers"
                        }

                        Table transactions {
                          transaction_id uuid [pk, not null]
                          account_id uuid [not null, ref: > accounts.account_id]
                          txn_type transaction_type [not null]
                          amount numeric(18, 2) [not null]
                          currency_code char(3)
                          transaction_date timestamp
                          description text

                          Indexes {
                            (account_id) [name: "idx_transactions_account_id"]
                            (transaction_date) [name: "idx_transactions_date"]
                          }

                          Note: "Transaction records for accounts"
                        }
                        :"""
            },
            {
                "type": "text",
                "text": f"print only the output {i}"
                }
            ]
        }
    ]

    # Tokenize input
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        generation = generation[0][input_len:]

    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)


Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Project BankingSystem {
  database_type: PostgreSQL
  note: "Comprehensive sample DBML schema"
}

Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}

```dbml
Project BankingSystem {
  database_type: PostgreSQL
  note: "Comprehensive sample DBML schema"
}

Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  al

In [ ]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}""",
"""Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  alias varchar(100)
  date_of_birth text
  date_of_incorporation text
  national_id text
  country_of_legal_formation varchar(100)
  country_of_birth varchar(100)
  country_of_principle_business varchar(100)
  country_of_primary_residence varchar(100)
  tax_id text
  age int4
  gender text
  ethnicity varchar(100)
  nationality varchar(100)
  citizenship1 varchar(100)
  citizenship2 varchar(100)
  native_language varchar(50)
  religion varchar(100)
  marital_status text
  dependents_count int4
  household_income numeric(15, 2)
  education_level text
  employment_status text
  occupation varchar(100)
  industry varchar(100)
  company_name varchar(255)
  years_with_employer int4
  job_role varchar(100)
  annual_income numeric(15, 2)
  primary_bank_relation text
  credit_score numeric(5, 2)
  home_ownership text
  residence_duration int4
  vehicle_ownership bool
  financial_dependents int4
  preferred_currency varchar(10)
  preferred_branch varchar(100)
  preferred_contact_method text
  social_media_presence jsonb
  risk_score numeric(5, 2)
  created_at timestamp
  updated_at timestamp

  Indexes {
    (company_name) [name: "idx_customer_company"]
    (credit_score) [name: "idx_customer_credit"]
    (last_name, first_name) [name: "idx_customer_name"]
    (national_id) [name: "idx_customer_national_id"]
    (risk_score) [name: "idx_customer_risk"]
    (tax_id) [name: "idx_customer_tax_id"]
  }
"""
]

for i in dbml_inputs:
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": "You are a helpful assistant that explains DBML Commands in descriptive way."}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Based on DBML explain in general sense what does the table and columns represent {i}"""}
            ]
        }
    ]

    inputs = processor.apply_chat_template(
            messages, add_generation_prompt=True, tokenize=True,
            return_dict=True, return_tensors="pt"
        ).to(model.device, dtype=torch.bfloat16)

    input_len = inputs["input_ids"].shape[-1]

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        generation = generation[0][input_len:]

    decoded = processor.decode(generation, skip_special_tokens=True)
    print(decoded)




Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Okay, let's break down this DBML definition for the `customer_address` table. I'll explain it in a way that focuses on what the table and its columns *represent* in a real-world scenario, rather than just the technical details.

**Overall Purpose of the Table: `customer_address`**

The `customer_address` table is designed to store information about the various addresses associated with a customer.  It's a crucial table for any system that needs to track where customers live, ship products, or send correspondence.  The `Note: "Customer addresses"` at the end is a helpful comment indicating the table's purpose.

**Columns and What They Represent:**

Let's go through each column and explain its meaning:

*   **`address_id text [pk, not null]`**:
    *   **Represents:** A unique identifier for each individual address record. Think of it as a primary key.  It's a text field (meaning it can hold letters, numbers, and symbols) and is guaranteed to be unique for every address.  `[pk]` signifie

In [1]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}""",
"""Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  alias varchar(100)
  date_of_birth text
  date_of_incorporation text
  national_id text
  country_of_legal_formation varchar(100)
  country_of_birth varchar(100)
  country_of_principle_business varchar(100)
  country_of_primary_residence varchar(100)
  tax_id text
  age int4
  gender text
  ethnicity varchar(100)
  nationality varchar(100)
  citizenship1 varchar(100)
  citizenship2 varchar(100)
  native_language varchar(50)
  religion varchar(100)
  marital_status text
  dependents_count int4
  household_income numeric(15, 2)
  education_level text
  employment_status text
  occupation varchar(100)
  industry varchar(100)
  company_name varchar(255)
  years_with_employer int4
  job_role varchar(100)
  annual_income numeric(15, 2)
  primary_bank_relation text
  credit_score numeric(5, 2)
  home_ownership text
  residence_duration int4
  vehicle_ownership bool
  financial_dependents int4
  preferred_currency varchar(10)
  preferred_branch varchar(100)
  preferred_contact_method text
  social_media_presence jsonb
  risk_score numeric(5, 2)
  created_at timestamp
  updated_at timestamp

  Indexes {
    (company_name) [name: "idx_customer_company"]
    (credit_score) [name: "idx_customer_credit"]
    (last_name, first_name) [name: "idx_customer_name"]
    (national_id) [name: "idx_customer_national_id"]
    (risk_score) [name: "idx_customer_risk"]
    (tax_id) [name: "idx_customer_tax_id"]
  }
"""
]

system_message = """You are a database documentation expert. Provide concise explanations of database tables and columns:
1. TABLE: 1 sentence real-world purpose
2. COLUMNS: Bullet points with:
   - Business meaning
   - Data type reason
   - Key constraints (PK/FK)
Keep each explanation 1 line, maximum 3 lines for complex columns."""

user_template = """Explain this database table in real-world terms:
{dbml_input}

Provide only the explanations, no DBML code."""

for dbml_input in dbml_inputs:
    # First convert your DDL to DBML (use your existing conversion code)
    #dbml_output = convert_to_dbml(dbml_input)

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Explain this database table in real-world terms:
{dbml_input}

Provide only the explanations, no DBML code."""
                }
            ]
        }
    ]

    # Generate and print only the explanation
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=1024, do_sample=False)
        explanation = processor.decode(generation[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

    print(explanation)
    print("\n" + "="*80 + "\n")




Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


This table stores multiple addresses associated with each customer, allowing for billing, shipping, and other address types.

*   **address_id:** Unique identifier for each address record - `text` (flexible, allows for UUIDs or other formats), `PK` (primary key, ensures uniqueness), `not null` (required).
*   **customer_id:** Links the address to a specific customer - `text` (matches customer ID format), `not null` (required), `FK` (foreign key referencing `customers.customer_id`, maintains referential integrity).
*   **address_type:** Specifies the purpose of the address (e.g., "billing", "shipping") - `text` (allows for various address types), `not null` (required).
*   **street:** The street address - `varchar(255)` (common address length), `not null` (required).
*   **city:** The city - `varchar(100)` (typical city name length), `not null` (required).
*   **state:** The state or province - `varchar(100)` (allows for longer state names), optional.
*   **postal_code:** The postal or 

In [1]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
warnings.filterwarnings("ignore")


# Constants
MODEL_ID = "google/gemma-3-12b-it"
GPU_COST_PER_HOUR = 1.89
SECONDS_PER_HOUR = 3600

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto"
).eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}""",
"""Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  alias varchar(100)
  date_of_birth text
  date_of_incorporation text
  national_id text
  country_of_legal_formation varchar(100)
  country_of_birth varchar(100)
  country_of_principle_business varchar(100)
  country_of_primary_residence varchar(100)
  tax_id text
  age int4
  gender text
  ethnicity varchar(100)
  nationality varchar(100)
  citizenship1 varchar(100)
  citizenship2 varchar(100)
  native_language varchar(50)
  religion varchar(100)
  marital_status text
  dependents_count int4
  household_income numeric(15, 2)
  education_level text
  employment_status text
  occupation varchar(100)
  industry varchar(100)
  company_name varchar(255)
  years_with_employer int4
  job_role varchar(100)
  annual_income numeric(15, 2)
  primary_bank_relation text
  credit_score numeric(5, 2)
  home_ownership text
  residence_duration int4
  vehicle_ownership bool
  financial_dependents int4
  preferred_currency varchar(10)
  preferred_branch varchar(100)
  preferred_contact_method text
  social_media_presence jsonb
  risk_score numeric(5, 2)
  created_at timestamp
  updated_at timestamp

  Indexes {
    (company_name) [name: "idx_customer_company"]
    (credit_score) [name: "idx_customer_credit"]
    (last_name, first_name) [name: "idx_customer_name"]
    (national_id) [name: "idx_customer_national_id"]
    (risk_score) [name: "idx_customer_risk"]
    (tax_id) [name: "idx_customer_tax_id"]
  }
"""
]

system_message = """You are a database documentation expert. Provide concise explanations of database tables and columns:
1. TABLE: 1 sentence real-world purpose
2. COLUMNS: Bullet points with:
   - Business meaning
   - Data type reason
   - Key constraints (PK/FK)
Keep each explanation in detail maximum upto 3 lines in a meaningful way."""

user_template = """Explain this database table in real-world terms:
{dbml_input}

Provide only the explanations, no DBML code."""

for dbml_input in dbml_inputs:
    # First convert your DDL to DBML (use your existing conversion code)
    #dbml_output = convert_to_dbml(dbml_input)

    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Explain this database table in real-world terms:
{dbml_input}

Provide only the explanations, no DBML code."""
                }
            ]
       you }
    ]

    # Generate and print only the explanation
    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)

    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        explanation = processor.decode(generation[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)

    print(explanation)
    print("\n" + "="*80 + "\n")




Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


**TABLE: customer_address**
This table stores multiple addresses associated with each customer, allowing for billing, shipping, and other address-related information.

**COLUMNS:**
*   **address_id:** Unique identifier for each address record. - *Data Type: text* - To accommodate potentially long, unique IDs. - *Key Constraint: PK*
*   **customer_id:** Links the address to a specific customer. - *Data Type: text* - Matches the customer identifier in the `customers` table. - *Key Constraint: FK referencing customers.customer_id*
*   **address_type:** Specifies the purpose of the address (e.g., "billing," "shipping," "home"). - *Data Type: text* - Allows for flexible address categorization. - *Key Constraint: Not Null*
*   **street:** The street address. - *Data Type: varchar(255)* - Sufficient length for most street addresses. - *Key Constraint: Not Null*
*   **city:** The city of the address. - *Data Type: varchar(100)* - Standard city name length. - *Key Constraint: Not Null*
*   **st

In [ ]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
from fpdf import FPDF
from fpdf.enums import XPos, YPos
import datetime
warnings.filterwarnings("ignore")

MODEL_ID = "google/gemma-3-12b-it"

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)


def format_explanation(explanation):
    lines = explanation.split('\n')
    formatted_lines = []
    
    for line in lines:
        line = line.replace('**', '').replace('*', '').strip()
        
        if line.startswith('Table:'):
            formatted_lines.append(('heading', line))
        elif line.startswith('Columns:'):
            formatted_lines.append(('subheading', line))
        elif ': ' in line:
            parts = [p.strip() for p in line.split('-') if p.strip()]
            if len(parts) > 1:
                formatted_lines.append(('column', parts[0]))
                for part in parts[1:]:
                    formatted_lines.append(('property', part))
            else:
                formatted_lines.append(('text', line))
        else:
            formatted_lines.append(('text', line))
    
    return formatted_lines

def generate_pdf(explanations, filename="dbml_explanations.pdf"):
    # Initialize PDF with proper margins
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    pdf.image("Cartograph_logo.png", x=170, y=10, w=30)
    
    # Title with CARTOGRAPH AI and DISCOVERY
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="CARTOGRAPH AI", align='C')
    pdf.ln(8)
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(0, 10, txt="DISCOVERY", align='C')
    pdf.ln(15)
    
    # Content with proper text handling
    pdf.set_font("Arial", size=11)
    
    for i, explanation in enumerate(explanations, 1):
        formatted = format_explanation(explanation)
        
        # Table number heading
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, txt=f"Table {i}")
        pdf.ln(8)
        
        for item_type, text in formatted:
            if item_type == 'heading':
                pdf.set_font("Arial", 'B', 12)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Reduced space after heading
            elif item_type == 'subheading':
                pdf.set_font("Arial", 'B', 11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(2)  # Reduced space after subheading
            elif item_type == 'column':
                pdf.set_font("Arial", 'B', 11)
                pdf.cell(5)  # Indent
                pdf.multi_cell(0, 6, txt=text)
                pdf.set_font("Arial", size=11)
                pdf.ln(1)  # Minimal space after column name
            elif item_type == 'property':
                pdf.cell(20)  # Indent
                pdf.multi_cell(0, 6, txt=f"- {text}")
                pdf.ln(1)  # Minimal space after property
            else:
                # Regular description text (not bold)
                pdf.set_font("Arial", size=11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Slightly more space after paragraphs
        
        pdf.ln(8)
        if i < len(explanations):
            pdf.add_page()
            pdf.set_left_margin(15)
            pdf.set_right_margin(15)
            # Add logo to subsequent pages
            try:
                pdf.image("logo.png", x=170, y=10, w=30)
            except:
                pass
    
    pdf.output(filename)
    print(f"PDF generated: {filename}")

explanations = []

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}""",
"""Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  alias varchar(100)
  date_of_birth text
  date_of_incorporation text
  national_id text
  country_of_legal_formation varchar(100)
  country_of_birth varchar(100)
  country_of_principle_business varchar(100)
  country_of_primary_residence varchar(100)
  tax_id text
  age int4
  gender text
  ethnicity varchar(100)
  nationality varchar(100)
  citizenship1 varchar(100)
  citizenship2 varchar(100)
  native_language varchar(50)
  religion varchar(100)
  marital_status text
  dependents_count int4
  household_income numeric(15, 2)
  education_level text
  employment_status text
  occupation varchar(100)
  industry varchar(100)
  company_name varchar(255)
  years_with_employer int4
  job_role varchar(100)
  annual_income numeric(15, 2)
  primary_bank_relation text
  credit_score numeric(5, 2)
  home_ownership text
  residence_duration int4
  vehicle_ownership bool
  financial_dependents int4
  preferred_currency varchar(10)
  preferred_branch varchar(100)
  preferred_contact_method text
  social_media_presence jsonb
  risk_score numeric(5, 2)
  created_at timestamp
  updated_at timestamp

  Indexes {
    (company_name) [name: "idx_customer_company"]
    (credit_score) [name: "idx_customer_credit"]
    (last_name, first_name) [name: "idx_customer_name"]
    (national_id) [name: "idx_customer_national_id"]
    (risk_score) [name: "idx_customer_risk"]
    (tax_id) [name: "idx_customer_tax_id"]
  }
"""
]

system_message = """You are a database documentation expert. Provide concise explanations of database tables and columns:
1. TABLE: 1 sentence real-world purpose
2. COLUMNS: Bullet points with:
   - Business meaning
   - Data type reason
   - Key constraints (PK/FK)
Keep each explanation in detail maximum upto 3 lines in a meaningful way."""

user_template = """Explain this database table in real-world terms:
{dbml_input}

Provide only the explanations, no DBML code."""

for dbml_input in dbml_inputs:
    
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Explain this database table in real-world terms:{dbml_input} Provide only the explanations, no DBML code."""
                }
            ]
        }
    ]

    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        explanation = processor.decode(generation[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    explanations.append(explanation)
    print(f"Processed table {len(explanations)}")

generate_pdf(explanations)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Processed table 1


In [1]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
from fpdf import FPDF
from fpdf.enums import XPos, YPos
import datetime
warnings.filterwarnings("ignore")

MODEL_ID = "google/gemma-3-12b-it"

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)


def format_explanation(explanation):
    lines = explanation.split('\n')
    formatted_lines = []
    
    for line in lines:
        line = line.replace('**', '').replace('*', '').strip()
        
        if line.startswith('Table:'):
            formatted_lines.append(('heading', line))
        elif line.startswith('Columns:'):
            formatted_lines.append(('subheading', line))
        elif ': ' in line:
            parts = [p.strip() for p in line.split('-') if p.strip()]
            if len(parts) > 1:
                formatted_lines.append(('column', parts[0]))
                for part in parts[1:]:
                    formatted_lines.append(('property', part))
            else:
                formatted_lines.append(('text', line))
        else:
            formatted_lines.append(('text', line))
    
    return formatted_lines

def generate_pdf(explanations, filename="dbml_explanations.pdf"):
    # Initialize PDF with proper margins
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    pdf.image("Cartograph_logo.png", x=170, y=10, w=30)
    
    # Title with CARTOGRAPH AI and DISCOVERY
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="CARTOGRAPH AI", align='C')
    pdf.ln(8)
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(0, 10, txt="DISCOVERY", align='C')
    pdf.ln(15)
    
    # Content with proper text handling
    pdf.set_font("Arial", size=11)
    
    for i, explanation in enumerate(explanations, 1):
        formatted = format_explanation(explanation)
        
        # Table number heading
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, txt=f"Table {i}")
        pdf.ln(8)
        
        for item_type, text in formatted:
            if item_type == 'heading':
                pdf.set_font("Arial", 'B', 12)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Reduced space after heading
            elif item_type == 'subheading':
                pdf.set_font("Arial", 'B', 11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(2)  # Reduced space after subheading
            elif item_type == 'column':
                pdf.set_font("Arial", 'B', 11)
                pdf.cell(5)  # Indent
                pdf.multi_cell(0, 6, txt=text)
                pdf.set_font("Arial", size=11)
                pdf.ln(1)  # Minimal space after column name
            elif item_type == 'property':
                pdf.cell(20)  # Indent
                pdf.multi_cell(0, 6, txt=f"- {text}")
                pdf.ln(1)  # Minimal space after property
            else:
                # Regular description text (not bold)
                pdf.set_font("Arial", size=11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Slightly more space after paragraphs
        
        pdf.ln(8)
        if i < len(explanations):
            pdf.add_page()
            pdf.set_left_margin(15)
            pdf.set_right_margin(15)
            # Add logo to subsequent pages
            try:
                pdf.image("logo.png", x=170, y=10, w=30)
            except:
                pass
    
    pdf.output(filename)
    print(f"PDF generated: {filename}")

explanations = []

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  street varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  postal_code varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}""",
"""Table customers {
  customer_id text [pk, not null]
  entity_individual varchar(20)
  legal_company_name varchar(100)
  company_trade_name varchar(100)
  full_legal_name varchar(100)
  name_prefix varchar(10)
  first_name varchar(100)
  middle_name varchar(100)
  last_name varchar(100)
  alias varchar(100)
  date_of_birth text
  date_of_incorporation text
  national_id text
  country_of_legal_formation varchar(100)
  country_of_birth varchar(100)
  country_of_principle_business varchar(100)
  country_of_primary_residence varchar(100)
  tax_id text
  age int4
  gender text
  ethnicity varchar(100)
  nationality varchar(100)
  citizenship1 varchar(100)
  citizenship2 varchar(100)
  native_language varchar(50)
  religion varchar(100)
  marital_status text
  dependents_count int4
  household_income numeric(15, 2)
  education_level text
  employment_status text
  occupation varchar(100)
  industry varchar(100)
  company_name varchar(255)
  years_with_employer int4
  job_role varchar(100)
  annual_income numeric(15, 2)
  primary_bank_relation text
  credit_score numeric(5, 2)
  home_ownership text
  residence_duration int4
  vehicle_ownership bool
  financial_dependents int4
  preferred_currency varchar(10)
  preferred_branch varchar(100)
  preferred_contact_method text
  social_media_presence jsonb
  risk_score numeric(5, 2)
  created_at timestamp
  updated_at timestamp

  Indexes {
    (company_name) [name: "idx_customer_company"]
    (credit_score) [name: "idx_customer_credit"]
    (last_name, first_name) [name: "idx_customer_name"]
    (national_id) [name: "idx_customer_national_id"]
    (risk_score) [name: "idx_customer_risk"]
    (tax_id) [name: "idx_customer_tax_id"]
  }
"""
]

system_message = """You are a database documentation expert. Provide concise explanations of database tables and columns:
1. TABLE: 1 sentence real-world purpose
2. COLUMNS: Bullet points with:
   - Business meaning
   - Data type reason
   - Key constraints (PK/FK)
Keep each explanation in 1 line and in a meaningful way."""

for dbml_input in dbml_inputs:
    
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Explain this database table in real-world terms:{dbml_input} Provide only the explanations, no DBML code."""
                }
            ]
        }
    ]

    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        explanation = processor.decode(generation[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    explanations.append(explanation)
    print(f"Processed table {len(explanations)}")

generate_pdf(explanations)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Processed table 1
Processed table 2
PDF generated: dbml_explanations.pdf


In [2]:
!pip install pandas --quiet

In [1]:
import time
import difflib
import torch
from transformers import AutoProcessor, Gemma3ForConditionalGeneration
import warnings
from fpdf import FPDF
from fpdf.enums import XPos, YPos
import datetime
import pandas as pd
warnings.filterwarnings("ignore")

MODEL_ID = "google/gemma-3-12b-it"

# Load model and processor
model = Gemma3ForConditionalGeneration.from_pretrained(
    MODEL_ID, device_map="auto").eval()
processor = AutoProcessor.from_pretrained(MODEL_ID)


def format_explanation(explanation):
    lines = explanation.split('\n')
    formatted_lines = []
    
    for line in lines:
        line = line.replace('**', '').replace('*', '').strip()
        
        if line.startswith('Table:'):
            formatted_lines.append(('heading', line))
        elif line.startswith('Columns:'):
            formatted_lines.append(('subheading', line))
        elif ': ' in line:
            parts = [p.strip() for p in line.split('-') if p.strip()]
            if len(parts) > 1:
                formatted_lines.append(('column', parts[0]))
                for part in parts[1:]:
                    formatted_lines.append(('property', part))
            else:
                formatted_lines.append(('text', line))
        else:
            formatted_lines.append(('text', line))
    
    return formatted_lines

def generate_pdf(explanations, filename="dbml_explanations.pdf"):
    # Initialize PDF with proper margins
    pdf = FPDF()
    pdf.set_auto_page_break(auto=True, margin=15)
    pdf.add_page()
    pdf.set_left_margin(15)
    pdf.set_right_margin(15)
    pdf.image("Cartograph_logo.png", x=170, y=10, w=30)
    
    # Title with CARTOGRAPH AI and DISCOVERY
    pdf.set_font("Arial", 'B', 16)
    pdf.cell(0, 10, txt="CARTOGRAPH AI", align='C')
    pdf.ln(8)
    pdf.set_font("Arial", 'B', size=12)
    pdf.cell(0, 10, txt="DISCOVERY", align='C')
    pdf.ln(15)
    
    # Content with proper text handling
    pdf.set_font("Arial", size=11)
    
    for i, explanation in enumerate(explanations, 1):
        formatted = format_explanation(explanation)
        
        # Table number heading
        pdf.set_font("Arial", 'B', 12)
        pdf.cell(0, 10, txt=f"Table {i}")
        pdf.ln(8)
        
        for item_type, text in formatted:
            if item_type == 'heading':
                pdf.set_font("Arial", 'B', 12)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Reduced space after heading
            elif item_type == 'subheading':
                pdf.set_font("Arial", 'B', 11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(2)  # Reduced space after subheading
            elif item_type == 'column':
                pdf.set_font("Arial", 'B', 11)
                pdf.cell(5)  # Indent
                pdf.multi_cell(0, 6, txt=text)
                pdf.set_font("Arial", size=11)
                pdf.ln(1)  # Minimal space after column name
            elif item_type == 'property':
                pdf.cell(20)  # Indent
                pdf.multi_cell(0, 6, txt=f"- {text}")
                pdf.ln(1)  # Minimal space after property
            else:
                # Regular description text (not bold)
                pdf.set_font("Arial", size=11)
                pdf.multi_cell(0, 6, txt=text)
                pdf.ln(4)  # Slightly more space after paragraphs
        
        pdf.ln(8)
        if i < len(explanations):
            pdf.add_page()
            pdf.set_left_margin(15)
            pdf.set_right_margin(15)
            # Add logo to subsequent pages
            try:
                pdf.image("logo.png", x=170, y=10, w=30)
            except:
                pass
    
    pdf.output(filename)
    print(f"PDF generated: {filename}")

explanations = []

dbml_inputs=[
    """Table customer_address {
  address_id text [pk, not null]
  customer_id text [not null, ref: > customers.customer_id]
  address_type text [not null]
  xmp_strt_vl varchar(255) [not null]
  city varchar(100) [not null]
  state varchar(100)
  xmt_pst_cd varchar(20)
  country varchar(100) [not null]
  is_primary bool [default: false, not null]

  Indexes {
    (customer_id) [name: "idx_customer_address_customer_id"]
    (address_type) [name: "idx_customer_address_type"]
  }

  Note: "Customer addresses"
}"""
]

df=pd.read_csv("Customer_address.csv")
data=df.head()

system_message = """You are a database documentation expert. Provide concise explanations of database tables and columns:
1. TABLE: 1 sentence real-world purpose
2. COLUMNS: Bullet points with:
   - Business meaning
   - Data type reason
   - Key constraints (PK/FK)
Keep each explanation in 1 line and in a meaningful way."""

for dbml_input in dbml_inputs:
    
    messages = [
        {
            "role": "system",
            "content": [{"type": "text", "text": system_message}]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": f"""Explain this database table in real-world terms:{dbml_input} and few columns are coded beacuse of privacy concers. So here's the data {data}
                    for reference.Provide only the explanations, no DBML code."""
                }
            ]
        }
    ]

    inputs = processor.apply_chat_template(
        messages, add_generation_prompt=True, tokenize=True,
        return_dict=True, return_tensors="pt"
    ).to(model.device, dtype=torch.bfloat16)
    
    with torch.inference_mode():
        generation = model.generate(**inputs, max_new_tokens=2048, do_sample=False)
        explanation = processor.decode(generation[0][inputs["input_ids"].shape[-1]:], skip_special_tokens=True)
    
    explanations.append(explanation)
    print(f"Processed table {len(explanations)}")

generate_pdf(explanations)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


Processed table 1
PDF generated: dbml_explanations.pdf
